In [34]:

#parses the log file and returns a dictionary with the key being the user's email and the value is a list of their sessions
def parse_log(file):
    user_log = {}
    fh = open(file, "r")
    text = fh.readlines()
    fh.close()
    for lines in text:
        items = lines.split()
        user_email = items[4]
        if user_email in user_log:
            user_log[user_email].append({"date": (" ").join([items[0], items[1]]), "activity": items[2], "server": items[3]})
        else:
            user_log[user_email] = [{"date": (" ").join([items[0], items[1]]), "activity": items[2], "server": items[3]}]
    return user_log

user_log = parse_log("userlog.log")
